In [ ]:
import numpy as np

In [ ]:
R,C = (2,8)
Ksplit=16
M=256
N=1024
K=512
A = np.random.rand(R,C,M//R,K//C)
B = np.random.rand(R,C,K//R,N//C)
#C = np.zeros([(R,C,M//R,N//C)])

In [ ]:
#A.shape = (R,C,M//R,K//C)
def allgather_row(A, grid, split, idx):
    splitsize = A.shape[3]//split
    A2 = np.zeros([grid[0], A.shape[2], grid[1],splitsize])
    A3 = A.reshape(A.shape[0],A.shape[1],A.shape[2],splitsize, split)
    for i in range(grid[0]):
        for j in range(grid[1]):
            A2[i,:,j] = A3[i,j,:,:,idx]
    return A2.reshape(grid[0], A.shape[2], grid[1]*splitsize)

#A.shape = (R,C,M//R,K//C)
def allgather_col(B, grid, split, idx):
    splitsize = B.shape[2]//split
    B2 = np.zeros([grid[1], grid[0], splitsize, B.shape[3]])
    B3 = B.reshape(B.shape[0], B.shape[1], splitsize, split, B.shape[3])
    for i in range(grid[0]):
        for j in range(grid[1]):
            B2[j,i] = B3[i,j,:,idx]
    return B2.reshape(grid[1], grid[0]*splitsize, B.shape[3])

rowTest = np.resize(np.arange(64),[64,64]).reshape(4,16,2,32).transpose(0,2,1,3)
result = allgather_row(rowTest,(4,2),8,0)
print(result[0])
print(result[0].shape)
colTest = np.resize(np.arange(64),[64,64]).transpose().reshape(4,16,2,32).transpose(0,2,1,3)
result = allgather_col(colTest,(4,2),8,0)
print(result[0])
print(result[0].shape)
            

In [ ]:
def PartialAllgatherOS(A,B,grid,split):
    #assert A.shape[3] == B.shape[2]
    R,C = grid
    print(R)
    print(C)
    AB = np.zeros([R,C,A.shape[2],B.shape[3]])
    for k in range(split):
        A2 = allgather_row(A,grid,split,k)
        B2 = allgather_col(B,grid,split,k)
        for i in range(R):
            for j in range(C):
                AB[i,j] = AB[i,j] + np.matmul(A2[i], B2[j])
    return AB
R,C = (2,8)
Ksplit=16
M=1024
N=1024
K=1024
X = np.random.randn(R,C,M//R,K//C)
W = np.random.randn(R,C,K//R,N//C)
#X = np.random.randint(0,10,[R,C,M//R,K//C])
#W = np.random.randint(0,10,[R,C,K//R,N//C])
Y1 = PartialAllgatherOS(X,W,(R,C),Ksplit)
X1 = X.transpose(0,2,1,3).reshape(M,K)
W1 = W.transpose(0,2,1,3).reshape(K,N)
Y2 = np.matmul(X1,W1)
Y11 = Y1.transpose(0,2,1,3).reshape(M,N)
np.abs(Y11-Y2).mean()


In [ ]:
from CostModel import *

def SOLCompareOS(B,I,O, N=8, flops=240*1024**4, bw=40*1024**3):   
    print("B: {}, I: {}, O:{}".format(B,I,O))
    myMesh = DeviceMesh((N,N), flops, bw, link_latency=8e-6) 
    rectangle = DeviceMesh((N*2,N//2), flops, bw) 

    print("Cannon {}x{}: ".format(N,N) + str(tup2ms(Cannon_OS(myMesh,B,O,I))))
    MeshTune(myMesh, GSPMD_OS, B,O,I)
    MeshTune(myMesh, SUMMA_OS, B,O,I)
    MeshTune(myMesh, Systolic_OS, B,O,I)
    if (min(B*I, B*O, I*O) == B*I):
        print("Wang, et.al, 2023 Isplit: " + str(tup2ms(LogicalRing_ISplit(myMesh, B, O, I))))
    elif (min(B*I, B*O, I*O) == O*I):
        print("Wang, et.al, 2023 Wsplit: " + str(tup2ms(LogicalRing_WSplit(myMesh, B, O, I))))
    else:
        print("Wang, et.al, 2023 Osplit: " + str(tup2ms(LogicalRing_OSplit(myMesh, B, O, I))))
SOLCompareOS(2*1*2048, 160*128, 4*160*128, N=2)
SOLCompareOS(2*4*2048, 160*128, 4*160*128, N=4)
SOLCompareOS(2*16*2048, 160*128, 4*160*128, N=8)
SOLCompareOS(2*64*2048, 160*128, 4*160*128, N=16)
SOLCompareOS(2*256*2048, 160*128, 4*160*128, N=32)
#SOLCompareOS(2*1024*2048, 160*128, 4*160*128, N=64)

In [ ]:
def SOLCompareOS2(B,I,O, shape, flops=240*1024**4, bw=40*1024**3):   
    print("B: {}, I: {}, O:{}".format(B,I,O))
    myMesh = DeviceMesh(shape, flops, bw, link_latency=8e-6) 

    MeshTune(myMesh, GSPMD_OS, B,O,I)
    MeshTune(myMesh, SUMMA_OS, B,O,I)
    MeshTune(myMesh, Systolic_OS, B,O,I)
    if (min(B*I, B*O, I*O) == B*I):
        print("Wang, et.al, 2023 Isplit: " + str(tup2ms(LogicalRing_ISplit(myMesh, B, O, I))))
    elif (min(B*I, B*O, I*O) == O*I):
        print("Wang, et.al, 2023 Wsplit: " + str(tup2ms(LogicalRing_WSplit(myMesh, B, O, I))))
    else:
        print("Wang, et.al, 2023 Osplit: " + str(tup2ms(LogicalRing_OSplit(myMesh, B, O, I))))

SOLCompareOS2(4*1*2048, 160*128, 4*160*128, shape=(4,2))
SOLCompareOS2(4*4*2048, 160*128, 4*160*128, shape=(8,4))
SOLCompareOS2(4*16*2048, 160*128, 4*160*128, shape=(16,8))
SOLCompareOS2(4*64*2048, 160*128, 4*160*128, shape=(32,16))
SOLCompareOS2(4*256*2048, 160*128, 4*160*128, shape=(64,32))

In [ ]:
from CostModel import *
def SOLCompareIS(B,I,O, N=8, flops=240*1024**4, bw=40*1024**3):   
    print("B: {}, I: {}, O:{}".format(B,I,O))
    myMesh = DeviceMesh((N,N), flops, bw, link_latency=8e-6) 
    rectangle = DeviceMesh((N*2,N//2), flops, bw) 

    print("Cannon {}x{}: ".format(N,N) + str(tup2ms(Cannon_IS(myMesh,B,O,I))))
    MeshTune(myMesh, GSPMD_IS, B,O,I)
    MeshTune(myMesh, SUMMA_IS, B,O,I)
    MeshTune(myMesh, Systolic_IS, B,O,I)
    if (min(B*I, B*O, I*O) == B*I):
        print("Wang, et.al, 2023 Isplit: " + str(tup2ms(LogicalRing_ISplit(myMesh, B, O, I))))
    elif (min(B*I, B*O, I*O) == O*I):
        print("Wang, et.al, 2023 Wsplit: " + str(tup2ms(LogicalRing_WSplit(myMesh, B, O, I))))
    else:
        print("Wang, et.al, 2023 Osplit: " + str(tup2ms(LogicalRing_OSplit(myMesh, B, O, I))))
#SOLCompareIS(2*1*2048, 4*160*128, 160*128, N=2)
SOLCompareIS(2*4*2048, 4*160*128, 160*128, N=4)
SOLCompareIS(2*16*2048, 4*160*128, 160*128, N=8)
SOLCompareIS(2*64*2048, 4*160*128, 160*128, N=16)
SOLCompareIS(2*256*2048, 4*160*128, 160*128, N=32)

In [ ]:
from CostModel import *
def SOLCompareWS(B,I,O, N=8, flops=240*1024**4, bw=40*1024**3):   
    print("B: {}, I: {}, O:{}".format(B,I,O))
    myMesh = DeviceMesh((N,N), flops, bw, link_latency=8e-6) 
    rectangle = DeviceMesh((N*2,N//2), flops, bw) 

    print("Cannon {}x{}: ".format(N,N) + str(tup2ms(Cannon_WS(myMesh,B,O,I))))
    MeshTune(myMesh, GSPMD_WS, B,O,I)
    MeshTune(myMesh, SUMMA_WS, B,O,I)
    MeshTune(myMesh, Systolic_WS, B,O,I)
    if (min(B*I, B*O, I*O) == B*I):
        print("Wang, et.al, 2023 Isplit: " + str(tup2ms(LogicalRing_ISplit(myMesh, B, O, I))))
    elif (min(B*I, B*O, I*O) == O*I):
        print("Wang, et.al, 2023 Wsplit: " + str(tup2ms(LogicalRing_WSplit(myMesh, B, O, I))))
    else:
        print("Wang, et.al, 2023 Osplit: " + str(tup2ms(LogicalRing_OSplit(myMesh, B, O, I))))
#SOLCompareIS(2*1*2048, 4*160*128, 160*128, N=2)
#SOLCompareWS(160*128, 4*160*128, 2*4*2048, N=4)
SOLCompareWS(160*128, 4*160*128, 2*16*2048, N=8)
SOLCompareWS(160*128, 4*160*128, 2*64*2048, N=16)
SOLCompareWS(160*128, 4*160*128, 2*256*2048, N=32)
SOLCompareWS(160*128, 4*160*128, 2*1024*2048, N=64)

In [ ]:
from CostModel import *
def AutotuneLayer(meshsize, B, I, O, algorithm='SUMMA', flops=240*1024**4, bw=40*1024**3,link_latency=3e-6, verbose=True, steps=32):
    
    if algorithm == 'SUMMA':
        funcs = {'OS':SUMMA_OS, 'IS':SUMMA_IS, 'WS':SUMMA_WS}
    elif algorithm == 'GSPMD':
        funcs = {'OS':GSPMD_OS, 'IS':GSPMD_IS, 'WS':GSPMD_WS}
    else:
        funcs = {'OS':Systolic_OS, 'IS':Systolic_IS, 'WS':Systolic_WS}
    
    meshshape = (2, meshsize//2)
    mesh = DeviceMesh(meshshape, flops, bw, link_latency)
    comm_fw, overlap_fw = funcs['OS'](mesh, B, I, O, steps=steps)
    comm_bd, overlap_bd = funcs['IS'](mesh, B, O, I, steps=steps)
    comm_bw, overlap_bw = funcs['WS'](mesh, I, O, B, steps=steps)
    bestcomm = (comm_fw + comm_bd + comm_bw)
    bestoverlap = (overlap_fw + overlap_bd + overlap_bw)
    
        
    bestshape = meshshape
    meshshape = (4, meshsize//4)
    while meshshape[1] >= 4:
        comm_fw, overlap_fw = funcs['OS'](mesh, B, I, O, steps=steps)
        comm_bd, overlap_bd = funcs['IS'](mesh, B, O, I, steps=steps)
        comm_bw, overlap_bw = funcs['WS'](mesh, I, O, B, steps=steps)
        comm = (comm_fw + comm_bd + comm_bw)
        overlap = (overlap_fw + overlap_bd + overlap_bw)
        if comm+overlap < bestcomm + bestoverlap:
            bestshape = meshshape
            bestcomm = comm
            bestoverlap = overlap
        #print("Best mesh for {} is {}x{}, result: {}".format(
        #    func.__name__,meshshape[0],meshshape[1],tup2ms(func(mesh, M, N, K))))
        meshshape = (meshshape[0]*2, meshshape[1]//2)
        mesh.reshape(meshshape)
        
    mesh.reshape(bestshape)
    if verbose:
        print("Best mesh for {} is {}x{}, result: ({:.4f}ms, {:.4f}ms)".format(
        algorithm,bestshape[0],bestshape[1],bestcomm*1000, bestoverlap*1000))
    return bestshape

def CannonLayer(N, B, I, O, flops=240*1024**4, bw=40*1024**3,link_latency=3e-6):
    
    meshshape = (N, N)
    mesh = DeviceMesh(meshshape, flops, bw, link_latency)
    comm_fw, overlap_fw = Cannon_OS(mesh, B, I, O)
    comm_bd, overlap_bd = Cannon_IS(mesh, B, O, I)
    comm_bw, overlap_bw = Cannon_WS(mesh, I, O, B)
    bestcomm = (comm_fw + comm_bd + comm_bw)
    bestoverlap = (overlap_fw + overlap_bd + overlap_bw)
    

    print("Cannon {}x{}, result: ({:.4f}ms, {:.4f}ms)".format(
        meshshape[0],meshshape[1],bestcomm*1000, bestoverlap*1000))

AutotuneLayer(16, 2*4*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(16, 2*4*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(16, 2*4*2048, 160*128, 4*160*128, algorithm='Ours')
CannonLayer(4, 2*4*2048, 160*128, 4*160*128)

#AutotuneLayer(32, 2*8*2048, 160*128, 4*160*128, algorithm='SUMMA')
#AutotuneLayer(32, 2*8*2048, 160*128, 4*160*128, algorithm='GSPMD')
#AutotuneLayer(32, 2*8*2048, 160*128, 4*160*128, algorithm='Ours')

AutotuneLayer(64, 2*16*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(64, 2*16*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(64, 2*16*2048, 160*128, 4*160*128, algorithm='Ours')
CannonLayer(8, 2*16*2048, 160*128, 4*160*128)

#AutotuneLayer(128, 2*32*2048, 160*128, 4*160*128, algorithm='SUMMA')
#AutotuneLayer(128, 2*32*2048, 160*128, 4*160*128, algorithm='GSPMD')
#AutotuneLayer(128, 2*32*2048, 160*128, 4*160*128, algorithm='Ours')

AutotuneLayer(256, 2*64*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(256, 2*64*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(256, 2*64*2048, 160*128, 4*160*128, algorithm='Ours', steps=16)
CannonLayer(16, 2*64*2048, 160*128, 4*160*128)

AutotuneLayer(1024, 2*256*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(1024, 2*256*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(1024, 2*256*2048, 160*128, 4*160*128, algorithm='Ours', steps=8)
CannonLayer(32, 2*256*2048, 160*128, 4*160*128)

AutotuneLayer(4096, 2*1024*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(4096, 2*1024*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(4096, 2*1024*2048, 160*128, 4*160*128, algorithm='Ours', steps=4)
CannonLayer(64, 2*1024*2048, 160*128, 4*160*128)

In [ ]:


AutotuneLayer(32, 2*8*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(32, 2*8*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(32, 2*8*2048, 160*128, 4*160*128, algorithm='Ours')



AutotuneLayer(128, 2*32*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(128, 2*32*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(128, 2*32*2048, 160*128, 4*160*128, algorithm='Ours')

AutotuneLayer(512, 2*128*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(512, 2*128*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(512, 2*128*2048, 160*128, 4*160*128, algorithm='Ours')

AutotuneLayer(2048, 2*512*2048, 160*128, 4*160*128, algorithm='SUMMA')
AutotuneLayer(2048, 2*512*2048, 160*128, 4*160*128, algorithm='GSPMD')
AutotuneLayer(2048, 2*512*2048, 160*128, 4*160*128, algorithm='Ours')


In [ ]:
bestshapes = {'SUMMA':{}, 'GSPMD':{}, 'Ours':{}}
cnt = 16
while cnt <= 65536:
    for alg in ['SUMMA', 'GSPMD', 'Ours']:
        bestshapes[alg][cnt] = AutotuneLayer(cnt, cnt*1024, 160*128, 4*160*128, algorithm=alg, verbose=False)
    cnt = cnt * 2
print(bestshapes)

In [ ]:
def Autotune3D(cubesize, B, I, O, algorithm='SUMMA', flops=240*1024**4, bw=40*1024**3,link_latency=3e-6, verbose=True):
    
    if algorithm == 'SUMMA':
        funcs = {'OS':SUMMA_OS, 'IS':SUMMA_IS, 'WS':SUMMA_WS}
    elif algorithm == 'GSPMD':
        funcs = {'OS':GSPMD_OS, 'IS':GSPMD_IS, 'WS':GSPMD_WS}
    else:
        funcs = {'OS':Systolic_OS, 'IS':Systolic_IS, 'WS':Systolic_WS}
    

    meshshape = bestshapes[algorithm][cubesize]
    mesh = DeviceMesh(meshshape, flops, bw, link_latency)
    comm_fw, overlap_fw = funcs['OS'](mesh, B, I, O)
    comm_bd, overlap_bd = funcs['IS'](mesh, B, O, I)
    comm_bw, overlap_bw = funcs['WS'](mesh, I, O, B)
    bestcomm = (comm_fw + comm_bd + comm_bw)
    bestoverlap = (overlap_fw + overlap_bd + overlap_bw)
    bestshape = meshshape

    fsdp = 2
    bestfsdp = fsdp
    while fsdp < cubesize//16:
        meshshape = bestshapes[algorithm][cubesize//fsdp]
        mesh.reshape(meshshape)
        weight_size = I*O*2//cubesize
        steps = fsdp-1
        time_per_step = weight_size / (mesh.bw*2) # bidirectional
        comm_fsdp = steps*time_per_step + mesh.link_latency*steps
        comm_fw, overlap_fw = funcs['OS'](mesh, B//fsdp, I, O)
        comm_bd, overlap_bd = funcs['IS'](mesh, B//fsdp, O, I)
        comm_bw, overlap_bw = funcs['WS'](mesh, I, O, B//fsdp)
        comm = (comm_fw + comm_bd + comm_bw)
        overlap_fw = max(overlap_fw, comm_fsdp)
        overlap_bd = max(overlap_bd, comm_fsdp)
        overlap_bw = max(overlap_bw, comm_fsdp)
        overlap = (overlap_fw + overlap_bd + overlap_bw)
        if comm+overlap < bestcomm + bestoverlap:
            bestshape = meshshape
            bestcomm = comm
            bestoverlap = overlap
            bestfsdp = fsdp
        fsdp = fsdp * 2
    
    if verbose:
        print("Best cube for {} is {}x{}x{}, result: ({:.4f}ms, {:.4f}ms)".format(
        algorithm,bestfsdp, bestshape[0],bestshape[1],bestcomm*1000, bestoverlap*1000))
    return (bestfsdp, bestshape[0], bestshape[1])

Autotune3D(4096, 2*1024*2048, 160*128, 4*160*128, algorithm='Ours')
Autotune3D(4096, 2*1024*2048, 160*128, 4*160*128, algorithm='SUMMA')
Autotune3D(4096, 2*1024*2048, 160*128, 4*160*128, algorithm='GSPMD')

#Autotune3D(256, 2*32*2048, 160*128, 4*160*128, algorithm='Ours')
#Autotune3D(256, 2*32*2048, 160*128, 4*160*128, algorithm='SUMMA')
#Autotune3D(256, 2*32*2048, 160*128, 4*160*128, algorithm='GSPMD')

Autotune3D(16384, 2*4096*2048, 160*128, 4*160*128, algorithm='Ours')
Autotune3D(16384, 2*4096*2048, 160*128, 4*160*128, algorithm='SUMMA')
Autotune3D(16384, 2*4096*2048, 160*128, 4*160*128, algorithm='GSPMD')

Autotune3D(4*16384, 2*4*4096*2048, 160*128, 4*160*128, algorithm='Ours')
Autotune3D(4*16384, 2*4*4096*2048, 160*128, 4*160*128, algorithm='SUMMA')
Autotune3D(4*16384, 2*4*4096*2048, 160*128, 4*160*128, algorithm='GSPMD')